In [1]:
import streamlit as st
from collections import Counter
from PIL import Image
import numpy as np
import pandas as pd
import json
import requests as r
import time
from datetime import datetime

In [2]:
# выбор даты начала

start_day = st.date_input(
     "Дата начала",)
st.write('Воронка с', start_day, 'числа')

2022-05-16 13:57:59.695 
  command:

    streamlit run /Users/arturfattahov/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [3]:
# выбор даты конца

end_day = st.date_input(
     "Дата конца",)
st.write('Воронка до', end_day, 'числа')

In [4]:
# загрузка событий из AppMetrica

# but = st.button("Загрузить данные")

# if but:
#     while True:
#         header={
#             'Content-Type': 'application/x-yametrika+json',
#             'Authorization': 'OAuth AQAEA7qjKZXKAAfhX_j1dIIUR059lNONgVpbkWk'
#         }
#         metrika_url = f'https://api.appmetrica.yandex.ru/logs/v1/export/events.json?application_id=4065523&date_since={start_day}%2000%3A00%3A00&date_until={end_day}%2023%3A59%3A59&date_dimension=default&use_utf8_bom=true&fields=appmetrica_device_id%2Ccity%2Cevent_name'

#         #st.spinner('Wait for it...')

#         res = r.get(metrika_url, headers=header)
#         if res.status_code == 200:
#             items = json.loads(res.text.lstrip('\ufeff'))
#             df = pd.DataFrame.from_dict(items['data'])
#             st.write(df)
#             break
#         time.sleep(300)

In [2]:
df = pd.read_csv('events (3).csv')

In [3]:
df['event_datetime'] = pd.to_datetime(df['event_datetime'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values(by=['appmetrica_device_id', 'event_datetime'])

In [4]:
df['diff'] = df.groupby('appmetrica_device_id')['event_datetime'].diff(1)

In [5]:
df.head(20)

,appmetrica_device_id,event_name,event_datetime,diff
7531,14515923324850227,all_tasks_list_screen,2022-05-04 18:47:55,NaT
7562,14515923324850227,registration_screen,2022-05-04 18:47:57,0 days 00:00:02
7567,14515923324850227,signup_button_click,2022-05-04 18:48:07,0 days 00:00:10
7536,14515923324850227,check_sms_code_screen,2022-05-04 18:48:08,0 days 00:00:01
7532,14515923324850227,all_tasks_list_screen,2022-05-04 18:48:55,0 days 00:00:47
7544,14515923324850227,go_to_task_from_the_list,2022-05-04 18:49:06,0 days 00:00:11
7568,14515923324850227,task_card_screen,2022-05-04 18:49:06,0 days 00:00:00
7537,14515923324850227,click_respond_task_screen,2022-05-04 18:49:09,0 days 00:00:03
7545,14515923324850227,go_to_task_from_the_list,2022-05-04 18:49:38,0 days 00:00:29
7569,14515923324850227,task_card_screen,2022-05-04 18:49:38,0 days 00:00:00


In [6]:
df.query('event_name == "registration_screen"').count()[0]

185

In [7]:
df = df.groupby('appmetrica_device_id')['event_name'].apply(list).to_frame().reset_index()

In [8]:
df.head()

,appmetrica_device_id,event_name
0,14515923324850227,"[all_tasks_list_screen, registration_screen, s..."
1,199154579309413360,"[all_tasks_list_screen, task_card_screen, task..."
2,202987147747599767,"[registration_screen, signup_button_click, che..."
3,282293181952709685,"[client_tasks_screen, clk_my_tasks_butt, my_ta..."
4,295551228895410848,"[all_tasks_list_screen, go_to_task_from_the_li..."


In [9]:
print(df['event_name'][0])

['all_tasks_list_screen', 'registration_screen', 'signup_button_click', 'check_sms_code_screen', 'all_tasks_list_screen', 'go_to_task_from_the_list', 'task_card_screen', 'click_respond_task_screen', 'go_to_task_from_the_list', 'task_card_screen', 'go_to_task_from_the_list', 'task_card_screen', 'click_respond_task_screen']


In [10]:
st.write("# Шаги воронки")

first_step = st.multiselect('What are your favorite colors', 
['Green', 'registration_screen', 'Red', 'Blue'], 
['registration_screen'])


second_step = st.multiselect('What are your favorite colors', 
['Green', 'in_app_communication', 'by_phone_communication', 'Blue'], 
['in_app_communication', 'by_phone_communication'])


third_step = st.multiselect('What are your favorite colors', 
['Green', 'Yellow', 'Red', 'create_choos_communication_type'], 
['create_choos_communication_type'])


fourth_step = st.multiselect('What are your favorite colors', ['fg'])






# fifth_step = st.multiselect('What are your favorite colors', ['Green', 'Yellow', 'Red', 'Blue'], ['Yellow', 'Red'])
# sixth_step = st.multiselect('Выберите событие для шестого шага',df.event_name.unique().tolist())
# seventh_step = st.multiselect('Выберите событие для седьмого шага',df.event_name.unique().tolist())


2022-05-16 16:42:19.662 
  command:

    streamlit run /Users/arturfattahov/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [11]:
# st.write("# Шаги воронки")

# first_step = st.multiselect('Выберите событие для первого шага',df.event_name.unique().tolist())
# second_step = st.multiselect('Выберите событие для второго шага',df.event_name.unique().tolist())
# third_step = st.multiselect('Выберите событие для третьего шага',df.event_name.unique().tolist())
# fourth_step = st.multiselect('Выберите событие для четвертого шага',df.event_name.unique().tolist())
# fifth_step = st.multiselect('Выберите событие для пятого шага',df.event_name.unique().tolist())
# sixth_step = st.multiselect('Выберите событие для шестого шага',df.event_name.unique().tolist())
# seventh_step = st.multiselect('Выберите событие для седьмого шага',df.event_name.unique().tolist())


In [12]:
steps = [first_step, second_step, third_step, fourth_step]

In [13]:
df.query('event_name == "registration_screen"').count()[0]

0

In [16]:
list_checking = []


for i in steps:
    if len(i) > 0:
        list_checking.append(i)
        print(df.query('event_name in @list_checking').count()[0])
        print(list_checking)

[['registration_screen']]
[['registration_screen'], ['in_app_communication', 'by_phone_communication']]
[['registration_screen'], ['in_app_communication', 'by_phone_communication'], ['create_choos_communication_type']]


In [155]:
#tra = df.groupby('appmetrica_device_id')['event_name'].apply(list).to_frame().reset_index()

In [156]:
#['registration_screen'][0] in tra.iloc[0]['event_name']

In [157]:
# list_checking = []


# for i in steps:
#     new_df = pd.DataFrame()
#     if len(i) > 0:
#         list_checking.append(i)
#         print(list_checking)
#         # if len(list_checking) == 1:
#         #     print(tra[list_checking[0] in tra.event_name].shape[0])